In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
import re

In [2]:
df=pd.read_csv("../data/twitterInsuranceData.csv")

In [3]:
df=df.sample(len(df),random_state=42)

In [4]:
df["label"].value_counts()

label
health    451
travel    313
life      287
motor     181
home      149
Name: count, dtype: int64

In [5]:
multilabel=MultiLabelBinarizer()
y=multilabel.fit_transform(df['label'])

In [6]:
one_hot_encoded_data = pd.get_dummies(df, columns = ['label'])

In [14]:
y=one_hot_encoded_data.drop(columns=["Text","Lang","Username","Name", "Location"])
y=y.drop(columns=["label_health"])
y

,label_home,label_life,label_motor,label_travel
309,False,True,False,False
741,False,False,True,False
265,False,True,False,False
823,False,False,False,False
778,False,False,False,True
...,...,...,...,...
1095,False,True,False,False
1130,False,False,False,True
1294,False,False,False,False
860,False,False,False,True


In [15]:
x=df.drop(columns=["Lang","Username","Name","label"])

In [16]:
tfidf=TfidfVectorizer(analyzer='word',max_features=25,ngram_range=(1,3),stop_words='english')
x=tfidf.fit_transform(df['Text'])

In [17]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=3)
xtrain.shape,xtest.shape,ytrain.shape,ytest.shape
lr=LogisticRegression(solver='liblinear',penalty='l2')
from sklearn.multiclass import OneVsRestClassifier

In [18]:
clf=OneVsRestClassifier(lr)
clf.fit(xtrain,ytrain)

OneVsRestClassifier(estimator=LogisticRegression(solver='liblinear'))

In [19]:
from sklearn.metrics import accuracy_score

In [20]:
accuracy_score(ytest,clf.predict(xtest))

0.7978339350180506

In [21]:
x=['can I get insurance on life']
xt=tfidf.transform(x)
clf.predict(xt)

array([[0, 1, 0, 0]])

In [23]:
import pickle as pkl
pkl.dump(clf,open('../models/insurance-classification_model.pkl','wb'))
pkl.dump(tfidf,open('../models/insurance_classification_tfidf.pkl','wb'))